In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from cf_model import MLPContentFilter
from dataset import ConFDataset, Encoder, split_data
import import_ipynb
import db
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

importing Jupyter notebook from db.ipynb
best_contentfiltermodel_checkpoint.pth	menv
best_model_checkpoint.pth		mlruns
best_model_state.pth			movie.py
cf_model.py				path_to_your_database.db
config.py				__pycache__
dataset.py				q.py
db.ipynb				README.md
done.txt				test.db
fdstests.ipynb				testsources.ipynb
filemanager.py				todo.txt
holocene				train_colfilter.ipynb
indie_letterboxd.db			train_confilter.ipynb
indie_letterboxd_v2.db			user.py
letterboxd				wandb
letterboxd.db				web_spider.py
main.ipynb


In [2]:
DB_NAME = 'indie_letterboxd_v2'

In [3]:
db.display_size(DB_NAME)

Users: 2400

Movies: 771

Reviews: 18504


In [4]:
db.show_tables(DB_NAME)

Genres
sqlite_sequence
Directors
Actors
Movies
Countries
MoviesGenres
MoviesActors
Users
Reviews


In [5]:
#movie_id, title, release_date, director_id, country_id
db.display_table('Movies',DB_NAME,5)

('a7b132b2-ba44-40aa-80ba-7854684f7469', 'Cadejo Blanco', '2021-09-12 00:00:00', '3d62a2c2-7770-483c-893d-a9bdb162bd69', 1)
('2570aedf-c152-4818-a57d-62b3b2ccc871', '40 Years a Prisoner', '2020-09-17 00:00:00', '6faab4b7-5f9b-4c5d-8715-988f15612615', 2)
('8265924a-4c3e-48be-a987-6b77423d7d61', 'Get the Hell Out', '2020-06-28 00:00:00', '01cbd45b-967c-42a9-8e13-8f03f69fbcec', 3)
('477b80c6-6eca-43cb-b45d-95eb25caf893', 'The Melting Creatures', '2022-05-23 00:00:00', '95e18c09-d78a-4f4d-af19-66a3dd014ecc', 4)
('e856777a-7010-44e8-bf47-4c0ff965a41a', 'Death of a Whistleblower', '2023-09-09 00:00:00', '8754ab6b-35fa-42fc-9e5d-50def00808d8', 5)


In [6]:
db.display_table('MoviesGenres',DB_NAME,5)

('a7b132b2-ba44-40aa-80ba-7854684f7469', 1)
('a7b132b2-ba44-40aa-80ba-7854684f7469', 2)
('2570aedf-c152-4818-a57d-62b3b2ccc871', 3)
('8265924a-4c3e-48be-a987-6b77423d7d61', 4)
('8265924a-4c3e-48be-a987-6b77423d7d61', 5)


In [7]:
db.display_table('Genres',DB_NAME,10)

(1, 'Crime')
(2, 'Drama')
(3, 'Documentary')
(4, 'Horror')
(5, 'Action')
(6, 'Comedy')
(7, 'Science Fiction')
(8, 'Thriller')
(9, 'Romance')
(10, 'Music')


In [8]:
db.display_table('Countries',DB_NAME,5)

(1, 'Guatemala')
(2, 'USA')
(3, 'Taiwan')
(4, 'Chile')
(5, 'South Africa')


In [9]:
db.display_table('Users',DB_NAME,5)

('e5b13caf-817e-4161-ad39-b4a25bf3d367', 'Matias Kivimäki')
('42e54389-1ba4-4a78-8e3c-856b71a92d12', 'Monica Johansson')
('d9b17abb-9b5d-42e2-8de7-c3e356797052', 'Արշավիր Դուդուկչյան')
('90fc05a6-a309-4ed9-82ab-5b2f1f97ce7f', 'रमेश दवाडी')
('62e6acdc-aa52-4b80-a025-bb3fc3685512', 'लिटन मानन्धर')


# Import from sqlite3

In [10]:
import sqlite3

# Connect to your database
conn = sqlite3.connect(DB_NAME+'.db')
cursor = conn.cursor()

# SQL query
query = '''
            SELECT 
                Users.user_id,
                Users.name AS user_name,
                GROUP_CONCAT(DISTINCT Genres.genre_id) AS genre_ids,
                GROUP_CONCAT(DISTINCT Genres.name) AS genre_names,
                Countries.country_id,
                Countries.country_name,
                Movies.movie_id,
                Movies.title AS movie_name,
                Reviews.rating
            FROM 
                Reviews
            JOIN
                Users ON Reviews.user_id = Users.user_id
            JOIN 
                Movies ON Reviews.movie_id = Movies.movie_id
            JOIN 
                Countries ON Movies.country_id = Countries.country_id
            LEFT JOIN 
                MoviesGenres ON Movies.movie_id = MoviesGenres.movie_id
            LEFT JOIN 
                Genres ON MoviesGenres.genre_id = Genres.genre_id
            GROUP BY 
                Reviews.review_id, Reviews.rating
            ORDER BY 
                Movies.movie_id, Users.user_id

        '''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Close the connection
conn.close()

results

[('0381a2ce-de6e-4a92-8117-8a290833c219',
  'Terry Roberson',
  '2,9',
  'Drama,Romance',
  29,
  'South Korea',
  '001e5226-4f9f-4e01-a824-016ddcd62aad',
  'Endings Beginnings',
  0.4),
 ('05f8c9fa-0bce-4185-aaf2-1361253cdeba',
  'Edward Roman',
  '2,9',
  'Drama,Romance',
  29,
  'South Korea',
  '001e5226-4f9f-4e01-a824-016ddcd62aad',
  'Endings Beginnings',
  0.4),
 ('0687954c-a123-43ff-8833-7d5d5c8413a1',
  'विद्या नाम',
  '2,9',
  'Drama,Romance',
  29,
  'South Korea',
  '001e5226-4f9f-4e01-a824-016ddcd62aad',
  'Endings Beginnings',
  0.82),
 ('0b24d5c6-59e4-4fa5-9433-dd00f714dd31',
  'Francis Robinson',
  '2,9',
  'Drama,Romance',
  29,
  'South Korea',
  '001e5226-4f9f-4e01-a824-016ddcd62aad',
  'Endings Beginnings',
  1),
 ('122337fe-0776-4fe9-b89e-aae1fb6b7430',
  'Г-жа Панда Плюнкова',
  '2,9',
  'Drama,Romance',
  29,
  'South Korea',
  '001e5226-4f9f-4e01-a824-016ddcd62aad',
  'Endings Beginnings',
  1),
 ('2e5828a9-27fc-4929-8980-8bde7191c174',
  'Suhana Dey',
  '2,9',


In [11]:
len(results)

18504

In [12]:
list_of_lists = [list(t) for t in results]

# Shuffle the list of lists
random.shuffle(list_of_lists)

# Convert the shuffled list of lists back to a list of tuples
data = [tuple(lst) for lst in list_of_lists]

len(data)

18504

In [13]:
#user,genre,country
users_data = [did[1] for did in data]
genres_data = [did[3].split(',') for did in data]#<---- need list type
countries_data = [did[5] for did in data]
ratings_data = [did[-1] for did in data]
genres_data

[['Comedy', 'Romance'],
 ['Documentary'],
 ['Drama'],
 ['Comedy', 'Adventure'],
 ['Drama'],
 ['Drama'],
 ['Drama'],
 ['Drama', 'Comedy'],
 ['Documentary'],
 ['Drama', 'War'],
 ['Crime', 'Comedy', 'Music'],
 ['Drama', 'Comedy', 'History'],
 ['Documentary'],
 ['Crime', 'Drama'],
 ['Drama', 'Thriller', 'Mystery'],
 ['Drama'],
 ['Drama'],
 ['Drama', 'Thriller', 'Fantasy'],
 ['Drama'],
 ['Drama'],
 ['Crime', 'Drama', 'Romance'],
 ['Documentary'],
 ['Comedy', 'Music'],
 ['Horror', 'Thriller', 'Fantasy', 'Mystery'],
 ['Drama', 'Action', 'Fantasy'],
 ['Documentary', 'History', 'War'],
 ['Drama'],
 ['Drama', 'Comedy'],
 ['Crime', 'Thriller'],
 ['Documentary'],
 ['Comedy'],
 ['Drama', 'Romance'],
 ['Crime', 'Drama'],
 ['Crime', 'Drama'],
 ['Drama', 'Comedy'],
 ['Drama'],
 ['Drama'],
 ['Documentary'],
 ['Drama'],
 ['Drama', 'Animation', 'History'],
 ['Drama', 'Comedy'],
 ['Crime', 'Drama', 'Thriller'],
 ['Drama', 'Mystery'],
 ['Drama', 'Science Fiction', 'Thriller'],
 ['Drama'],
 ['Drama', 'Comed

In [14]:
#deal with genres..... make it a list????
encoder = Encoder(users=users_data,genres=genres_data,countries=countries_data)

In [15]:
encoder.one_hot_encode(['Drama', 'TV Movie'],'genres')

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1.])

In [16]:
encoder.encode('Gaičiūnas, Milana','users')

In [17]:
encoder.vocab_to_idx['genres']

{'Comedy': 0,
 'Romance': 1,
 'Documentary': 2,
 'Drama': 3,
 'Adventure': 4,
 'War': 5,
 'Crime': 6,
 'Music': 7,
 'History': 8,
 'Thriller': 9,
 'Mystery': 10,
 'Fantasy': 11,
 'Horror': 12,
 'Action': 13,
 'Animation': 14,
 'Science Fiction': 15,
 'Western': 16,
 'Family': 17,
 'TV Movie': 18}

In [18]:
encoder.one_hot_encode(['Documentary','War','Thriller'],'genres')

tensor([0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

In [19]:
datas = [(did[1],did[3].split(','),did[5],did[-1]) for did in data]
datas[7]

('Dani Klomp Jan', ['Drama', 'Comedy'], 'State of Palestine', 0.5)

In [20]:
all_ds = ConFDataset(datas,encoder)

In [21]:
all_ds.country_data[7]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
encoder.vocab_to_idx['genres']

{'Comedy': 0,
 'Romance': 1,
 'Documentary': 2,
 'Drama': 3,
 'Adventure': 4,
 'War': 5,
 'Crime': 6,
 'Music': 7,
 'History': 8,
 'Thriller': 9,
 'Mystery': 10,
 'Fantasy': 11,
 'Horror': 12,
 'Action': 13,
 'Animation': 14,
 'Science Fiction': 15,
 'Western': 16,
 'Family': 17,
 'TV Movie': 18}

In [23]:
all_ds.country_data[7]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
all_ds.genre_data[7]

tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

In [25]:
train_data, test_data = split_data(datas)
train_data, validation_data = split_data(train_data)

In [26]:
print(len(test_data))

3701


In [27]:
train_ds = ConFDataset(train_data,encoder)
test_ds = ConFDataset(test_data,encoder)
validation_ds = ConFDataset(validation_data,encoder)

In [28]:
len(test_ds)

3701

In [29]:
BATCH_SIZE = 64
train_data_loader = DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
validation_data_loader = DataLoader(validation_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
test_data_loader = DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)

In [30]:
all_ds[7]

(tensor(7),
 tensor([1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]),
 tensor(0.5000))

In [31]:
# for ds in test_data_loader:
#     print(ds)

In [32]:

num_users = len(encoder.vocab_to_idx['users'])
print(num_users)

num_mv_features = len(all_ds[0][1])
print(num_mv_features)

FEATURES = 700
model = MLPContentFilter(num_users, num_mv_features, embedding_dim=FEATURES)

LEARNING_RATE = 0.001
criterion = nn.MSELoss()
#weight decay L2 regularization
# optimiser = optim.SGD(model.parameters(), lr=0.001,weight_decay=1e-5)\
L2_REGULARIZATION = 0.1
optimiser = optim.SGD(model.parameters(), lr=LEARNING_RATE,weight_decay=L2_REGULARIZATION)
EPOCHS = 1000

2394
91


In [33]:
import mlflow
from mlflow.tracking import MlflowClient

# Optional: Set MLflow experiment
mlflow.set_experiment("Finetuned Content Filter")

# Log model parameters (example)
mlflow.start_run()
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("optimiser", type(optimiser).__name__)
mlflow.log_param("learning rate", LEARNING_RATE)
mlflow.log_param("batch size", BATCH_SIZE)
mlflow.log_param("L2 regularization", L2_REGULARIZATION)
mlflow.log_param("drop out", 0.5)

0.5

# PRETEST

In [34]:
#test data
model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_features, ratings in test_data_loader:
        
        predictions = model(user_ids,movie_features)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
print(test_loss)
mlflow.log_metric('pretraining test loss',test_loss)

0.31684375332112896


## wandb

In [37]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    entity="jcrich",
    project="content-filter",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "architecture": "content filter",
    "dataset": "imdb",
    "epochs": EPOCHS,
    }
)


In [36]:
# Initialize lists to keep track of losses and epochs

train_losses = []
validation_losses = []
epochs = []

#get it at it's finest....
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    total_epoch_loss = 0
    for batch_idx, (user_ids, movie_features, ratings) in tqdm(enumerate(train_data_loader), total=len(train_data_loader), desc=f'Epoch {epoch}'):

        optimiser.zero_grad()
        predictions = model(user_ids, movie_features)
        loss = criterion(predictions, ratings)

        loss.backward()
        optimiser.step()
        
        batch_loss = loss.item()
        total_epoch_loss += batch_loss
        
    # After all batches, calculate average loss for the epoch
    avg_epoch_loss = total_epoch_loss / len(train_data_loader)
    train_losses.append(avg_epoch_loss)
    epochs.append(epoch)  # Append the current epoch number
    
    wandb.log({"Training Loss": avg_epoch_loss, "Epoch": epoch})
    mlflow.log_metric("Training Loss", avg_epoch_loss, step=epoch)
    # print(f'Epoch: {epoch}, Average Training Loss: {avg_epoch_loss}')

    # Start validation phase
    model.eval()  # Set the model to evaluation mode
    total_validation_loss = 0
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        for batch_idx, (user_ids, movie_features, ratings) in enumerate(validation_data_loader):
            predictions = model(user_ids, movie_features)
            loss = criterion(predictions, ratings)
            total_validation_loss += loss.item()

    
    
    # Calculate and log validation loss after the epoch
    avg_validation_loss = total_validation_loss / len(validation_data_loader)
    validation_losses.append(avg_validation_loss)

    # Save the model if this epoch has the best validation loss so far
    if avg_validation_loss < best_val_loss:
        best_val_loss = avg_validation_loss
        mlflow.log_metric("Best Validation Loss", best_val_loss, step=epoch)
        mlflow.pytorch.log_model(model, "model")
        # Save model state
        torch.save(model.state_dict(), 'best_model_state.pth')
        # If you also want to save the optimizer state along with the model:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimiser_state_dict': optimiser.state_dict(),
            'loss': avg_validation_loss,
        }, 'best_contentfiltermodel_checkpoint.pth')
        # Log the model checkpoint as an artifact
        mlflow.log_artifact('best_contentfiltermodel_checkpoint.pth')
    
    wandb.log({"Validation Loss": avg_validation_loss, "Epoch": epoch})
    mlflow.log_metric("Validation Loss", avg_validation_loss, step=epoch)
    # print(f'Epoch: {epoch}, Average Validation Loss: {avg_validation_loss}')    
    # After training and validation phases, log the losses with the epoch number
    wandb.log({"Epoch": epoch, "Training Loss": avg_epoch_loss, "Validation Loss": avg_validation_loss})

wandb.finish()

Epoch 999: 100%|█████████████████████████████| 185/185 [00:00<00:00, 190.37it/s]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Training Loss,█▄▃▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Validation Loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,999
Training Loss,0.06003
Validation Loss,0.06109


In [38]:
#test data
model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_features, ratings in test_data_loader:
        predictions = model(user_ids,movie_features)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
print(test_loss)
mlflow.log_metric('evaluation posttraining test loss',test_loss)
mlflow.end_run()

0.06829113513231277


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


## Preference ranking

In [37]:
target_user_id = 140440102666832

In [38]:
e_uid = encoder.encode(target_user_id,encoder.user_to_idx)

AttributeError: 'Encoder' object has no attribute 'user_to_idx'

In [ ]:
def get_user_features(uid):

    for i in range(0,len(all_ds)):

        u,m,r = all_ds[i]
        # print(u)
        if u == uid:
            yield [m,r]
    

In [ ]:
def predict_ratings(e_uid,e_mid,model):
    e_uid_tensor = torch.tensor(e_uid, dtype=torch.int64).unsqueeze(0)
    movie_eid_tensor = torch.tensor(e_mid,dtype=torch.int64).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        user_rating = model(e_uid_tensor,movie_eid_tensor)
        return movie_eid_tensor,user_rating
        
    

In [ ]:
#get user and features
seen_features = set(tuple(fe) for fe in get_user_features(e_uid)) # Get user features if needed
seen_movies = set(sf[0].item() for sf in seen_features)
all_movies = set([num for num in range(30)])
unseen_movies = all_movies - seen_movies
unseen_features = {tuple(predict_ratings(e_uid,um,model)) for um in unseen_movies}
all_features = seen_features | unseen_features

sorted_set = sorted(all_features, key=lambda x: x[1])
top_n = 10
recommendations = list(filter(lambda x: x[0].item() in unseen_movies,sorted_set))
n_recommendations = list(map(lambda x: x[0],recommendations[-1:top_n*-1:-1]))
n_recommendations


# Matching

In [ ]:
#all users, with all movie ratings.
def generate_all_ratings(model,num_users,num_movies):
    #build matrix

    #data already encoded....
    all_data = [['dummy',mnm,unm,'dummy','0/10'] for unm in range(0,num_users) for mnm in range(0,num_movies)]
    # print(all_data)
    # print([em for em in all_data])
    fake_encoder = Encoder([did[2] for did in all_data],[did[1] for did in all_data])
    all_ds = CFDataset(all_data,fake_encoder)
    
    data_loader = DataLoader(all_ds, batch_size=1)
    for uid,mid,ra in data_loader:
        # print(uid,mid)
        prediction = model(uid,mid)
        yield (uid.item(),mid.item(),prediction.item())



In [ ]:
def create_ratings_tensor(data, num_users, num_movies):
    # Initialize an empty tensor to hold the ratings
    ratings_tensor = torch.zeros(num_users, num_movies)
    
    # Iterate over the data and fill the tensor
    for entry in data:
        user_id, movie_id, rating = entry
        # Convert rating to float
        rating = float(rating)
        ratings_tensor[user_id, movie_id] = rating
    
    return ratings_tensor

In [ ]:
# Step 1: Generate Predictions
predictions = [ra for ra in generate_all_ratings(model,num_users,num_movies)]
# print(predictions)
ratings_tensor = create_ratings_tensor(predictions,num_users,num_movies)
print(ratings_tensor)
# predictions[0]

# ratings_tensor.shape

# ratings_tensor[0]

import torch.nn.functional as F

# Assuming predicted_ratings is your tensor of predicted ratings
# predicted_ratings.shape should be (num_users, num_movies)

# # Calculate cosine similarity
# user_similarities = F.cosine_similarity(ratings_tensor, ratings_tensor, dim=1)

# user_similarities

## COSINE SIMILARITY

In [ ]:
# Step 1: Normalize predicted ratings
normalized_ratings = F.normalize(ratings_tensor, p=2, dim=1)

# Step 2: Calculate cosine similarity
user_similarities = torch.matmul(normalized_ratings, normalized_ratings.T)

# Set diagonal elements to a large negative value to exclude self-similarity
user_similarities.fill_diagonal_(-float('inf'))

# You can optionally convert the similarities tensor to a numpy array for easier manipulation
user_similarities_np = user_similarities.numpy()

# Print or use the user similarities tensor
print(user_similarities)

### Matches

In [ ]:
top_matches = {}
for i in range(len(user_similarities)):
    # Sort similarities for the current user i
    ranked_users = torch.argsort(torch.tensor(user_similarities[i]), descending=True)
    # Exclude self from top matches (optional)
    top_matches[i] = ranked_users[ranked_users != i]


In [ ]:
top_matches

In [ ]:
encoder.decode(71,encoder.idx_to_user)

In [ ]:
#decoded

decoded_matches = {encoder.decode(k,encoder.idx_to_user):[encoder.decode(ve,encoder.idx_to_user) for ve in v.tolist()] for k,v in top_matches.items()}
decoded_matches

In [ ]:
# Step 4: Select Top Matches
N = 5  # Number of top matches to select
for user, similar_users in decoded_matches.items():
    decoded_matches[user] = similar_users[:N]

decoded_matches

In [ ]:
#decode....

## FINALLY FIND THE MATCHES....

In [ ]:
target_user = 140440102666832



In [ ]:
decoded_matches[target_user]

In [ ]:
neighbour = 140440115331424

## Now see how they relate to the database...

In [ ]:
reviews = db.get_table_values('Reviews')

In [ ]:
#TODO

In [ ]:
def reviews_info(user_id,reviews):
    # print(db.get_table_values('Users'))
    info = {}
    info[user_id] = list(filter(lambda x :x[0] == user_id,db.get_table_values('Users')))[0][1]

    
    #TODO finish work...
    mov_tab = db.get_table_values('Movies')
    
    for review in reviews:
        mov_id = review[1]
        info[mov_id] = {'title':list(filter(lambda x :x[0] == mov_id,mov_tab))[0][1],'rating':review[4]}
        
    return info
    
    

In [ ]:
target_reviews = list(filter(lambda rv: rv[2] == target_user,reviews))
neighbour_reviews = list(filter(lambda rv: rv[2] == neighbour,reviews))

In [ ]:
tar_info = reviews_info(target_user,target_reviews)
neigh_info = reviews_info(neighbour,neighbour_reviews)

In [ ]:
tar_info

In [ ]:
neigh_info

# SAVE_MODEL

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')